In [113]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [114]:
### Load the trained model, scaler pickle,onehot
# model=load_model('model.h5')
with open('ensemble_model_SD.pkl','rb') as file:
    ensemble_model=pickle.load(file)

## load the encoder and scaler
with open('..\onehot_encoder_SD.pkl','rb') as file:
    onehot_encoder=pickle.load(file)

with open('..\label_encoder_SD.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

with open('..\scaler_SD.pkl', 'rb') as file:
    scaler = pickle.load(file)

<>:7: SyntaxWarning: invalid escape sequence '\o'
<>:10: SyntaxWarning: invalid escape sequence '\l'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\o'
<>:10: SyntaxWarning: invalid escape sequence '\l'
<>:13: SyntaxWarning: invalid escape sequence '\s'
C:\Users\0027OA744\AppData\Local\Temp\ipykernel_22288\2941454684.py:7: SyntaxWarning: invalid escape sequence '\o'
  with open('..\onehot_encoder_SD.pkl','rb') as file:
C:\Users\0027OA744\AppData\Local\Temp\ipykernel_22288\2941454684.py:10: SyntaxWarning: invalid escape sequence '\l'
  with open('..\label_encoder_SD.pkl', 'rb') as file:
C:\Users\0027OA744\AppData\Local\Temp\ipykernel_22288\2941454684.py:13: SyntaxWarning: invalid escape sequence '\s'
  with open('..\scaler_SD.pkl', 'rb') as file:


In [123]:
# Example input data
# input_data = {
#     'CreditScore': 600,
#     'Geography': 'France',
#     'Gender': 'Male',
#     'Age': 40,
#     'Tenure': 3,
#     'Balance': 60000,
#     'NumOfProducts': 2,
#     'HasCrCard': 1,
#     'IsActiveMember': 1,
#     'EstimatedSalary': 50000
# }

data = {
    "age": 48,
    "bp": 80,
    "sg": 1.02,
    "al": 1,
    "sugar": 0,
    # "rbc": ?,
    "pc": 'normal',
    "pcc": 'notpresent',
    "ba": 'notpresent',
    "bgr": 121,
    "bu": 36,
    "sc": 1.2,
    "sod": 111,
    "pot": 2.5,
    "hemo": 15.4,
    "pcv": 44,
    "wc": 7800,
    # "rc": 5.2,
    # "htn": 'yes',
    "dm": 'yes',
    "cad": 'no',
    "appet": 'good',
    "pe": 'no',
    "ane": 'no'
}

input_df = pd.DataFrame([data])

In [120]:
onehot_encoder.categories_

[array(['good', 'no', 'poor'], dtype=object),
 array(['good', 'no', 'yes'], dtype=object)]

In [124]:
# Select columns to encode
columns_to_encode = ['appet', 'pe']

# Fit and transform the categorical columns
encoded_array = onehot_encoder.transform(input_df[columns_to_encode])

# Convert the encoded array to a DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=onehot_encoder.get_feature_names_out(columns_to_encode))

# Drop original categorical columns and concatenate the new one-hot encoded DataFrame
input_df = input_df.drop(columns_to_encode, axis=1)
input_df = pd.concat([input_df, encoded_df], axis=1)

print(input_df.columns)


Index(['age', 'bp', 'sg', 'al', 'sugar', 'pc', 'pcc', 'ba', 'bgr', 'bu', 'sc',
       'sod', 'pot', 'hemo', 'pcv', 'wc', 'dm', 'cad', 'ane', 'appet_no',
       'appet_poor', 'pe_no', 'pe_yes'],
      dtype='object')


In [125]:
## Encode other categorical variables

columns_to_encode = ['pc', 'pcc', 'ba', 'dm', 'cad', 'ane']
# input_df['pc']=label_encoder.transform(input_df['pc'])

for col in columns_to_encode:
    # print(col)
    input_df[col] = label_encoder[col].transform(input_df[col])


input_df

,age,bp,sg,al,sugar,pc,pcc,ba,bgr,bu,...,hemo,pcv,wc,dm,cad,ane,appet_no,appet_poor,pe_no,pe_yes
0,48,80,1.02,1,0,1,0,0,121,36,...,15.4,44,7800,1,0,0,0.0,0.0,1.0,0.0


In [133]:
input_df = input_df[scaler.feature_names_in_]

In [134]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.19247086,  0.34434682,  0.39631199,  0.14698114, -0.35355339,
        -0.3346643 , -0.39193566, -0.40499449, -4.32281705, -3.26500669,
         1.06712249,  0.61094592, -0.22986653,  0.4404882 , -0.31767504,
        -0.22941573,  1.4010578 , -0.34099717, -0.43372666, -0.19738551,
        -0.51298918,  0.51946248, -0.46056619]])

In [138]:
ensemble_model.predict_

[('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
 ('xgb',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric='logloss',
                feature_types=None, feature_weights=None, gamma=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.1, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=3, max_leaves=None,
                min_child_weight=3, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=50, n_jobs=None,
                num_parallel_tree=None, ...)),
 ('cat', <catboost.core.CatBoostClassifier at 0x10bc56b3620>)]

In [140]:
## PRedict cdk
prediction = ensemble_model.predict(input_scaled)
prediction

array([1])

In [141]:
if prediction ==1 :
    print('The patient is likely to have chronic kidney disease')
else:
    print('The patient is NOT likely to have chronic kidney disease')

The patient is likely to have chronic kidney disease
